In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [ ]:
%load_ext qat.core.magic

In [ ]:
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()    

## 1. Data Discretization

In [ ]:
from AuxiliarFunctions import  get_histogram, PostProcessResults, TestBins, RunJob
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 4
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

## 2. Complete Data Loading

First of all we need to loading the complete data into the quantum state:

$$|\Psi\rangle_{n+1}=\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)|0\rangle_{n}\otimes|0\rangle_{1}$$

From module **QuantumMultiplexors_Module_examples** fucntion **LoadingData** can be used. This function receives numpy arrays with probability and function and outputs the quantum program for loading complete data and the mandatory $\mathcal{P}$ y $\mathcal{R}$ gates to perform the loading.

In [ ]:
from QuantumMultiplexors_Module_examples import ExpectationLoadingData

In [ ]:
qLprog, P_Gate, R_gate = ExpectationLoadingData(p_X, f_X)

In [ ]:
%qatdisplay P_Gate
%qatdisplay R_gate

circuit = qLprog.to_circ()
%qatdisplay circuit

In [ ]:
job = circuit.to_job(qubits=[n_qbits])
result = RunJob(lineal_qpu.submit(job))
LResults = PostProcessResults(result)
#Check
print(np.isclose(LResults['Probability'][1], sum(p_X*f_X)))

## 3. Amplificación de Amplitud

The quantum state resulting from the loading proccess $|\Psi\rangle_{n+1}$ can be expressed as a linear combination of twor othogonal states $|\Psi_{1}\rangle$ y $|\Psi_{0}\rangle$:

$$|\Psi\rangle_{n+1}=\sqrt{a}|\Psi_{1}\rangle+\sqrt{1-a}|\Psi_{0}\rangle$$

Where 

$$|\Psi_{1}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\frac{1}{\sqrt{a}}\sqrt{p(x)f(x)}|1\rangle$$
$$|\Psi_{0}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\frac{1}{\sqrt{1-a}}\sqrt{p(x)(1-f(x))}|0\rangle$$


The idea behind Quantum Phase Amplification is increase the probability of the $|\Psi_{1}\rangle$  using a strategy based on the Groover Algorithm. 

An Operator $\hat{Q}$ should be applied an optimal number of times $k$ in order to maximize probability of measure $|\Psi_{1}\rangle$. This operator is: 

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

Where $\hat{U}_{|\Psi_{2}\rangle}$ y $\hat{U}_{|\Psi_{0}\rangle}$ are:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$
$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle \langle \Psi|$$

Module **PhaseAmplification_Module** have all the functions in order to create properly gates for this operators

### 3.1 Operador $\hat{U}_{|\Psi_{0}\rangle}$

First mandatory operator is:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$

Equivalent circuit for the operator is:

$$\hat{U}_{|\Psi_{0}\rangle }=(\hat{I_{n}}\otimes X)(\hat{I_{n}}\otimes Z)(\hat{I_{n}}\otimes X)$$

Apply this operator on state $|\Psi\rangle_{n+1}$:

$$\hat{U}_{|\Psi_{0}\rangle} |\Psi\rangle_{n+1} = \sqrt{a}|\Psi_{1}\rangle-\sqrt{1-a}|\Psi_{0}\rangle$$

So operator $\hat{U}_{|\Psi_{0}\rangle }$ do an reflection operation around the axis defined by the state $|\Psi_{1}\rangle$


Abstract Gate **U_Phi_0** from **PhaseAmplification_Module** allow us do the operation:

In [ ]:
qLprog, P_Gate, R_gate = ExpectationLoadingData(p_X, f_X)
#Initial State
circuit = qLprog.to_circ()
job = circuit.to_job()
result = RunJob(lineal_qpu.submit(job))
InitialState = PostProcessResults(result)
#Applied Gate
from PhaseAmplification_Module import UPhi0_Gate
U_Phi_0_Gate = UPhi0_Gate(qLprog.qbit_count)
qLprog.apply(U_Phi_0_Gate, qLprog.registers)

%qatdisplay U_Phi_0_Gate --depth 1
circuit = qLprog.to_circ()
%qatdisplay circuit

job = circuit.to_job()
result = RunJob(lineal_qpu.submit(job))
UPhi0_State = PostProcessResults(result)

In order to test the operator the states with final qbit $|0\rangle$ ($|\Psi_{0}\rangle$) change the sign with respect to the initial state $|\Psi\rangle_{n+1}$ meanwhile states with final qbit in $|1\rangle$ ($|\Psi_{1}\rangle$) are the same that in the initial state $|\Psi\rangle_{n+1}$

In [ ]:
#Testing Final qbit |0> should be of different sign
LastQbit0 = np.isclose(
    InitialState.loc[[i for i in range(0, len(InitialState), 2)]]['Amplitude'], 
    -UPhi0_State.loc[[i for i in range(0, len(InitialState), 2)]]['Amplitude']
).all()
#Testing Final qbit |1> should be of same sign
LastQbit1 = np.isclose(
    InitialState.loc[[i for i in range(1, len(InitialState), 2)]]['Amplitude'], 
    UPhi0_State.loc[[i for i in range(1, len(InitialState), 2)]]['Amplitude']
).all()

print(LastQbit0 and LastQbit1)

### 3.2 Operador $\hat{U}_{|\Psi\rangle}$

Operator $\hat{U}_{|\Psi\rangle}$ is based in Groover's difusor:

$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle \langle \Psi|$$

Additionally we know that:

$$|\Psi\rangle_{n+1}=\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)|0\rangle_{n}\otimes|0\rangle_{1}$$

Then the operator can be decomposed in the following way:

$$\hat{U}_{|\Psi\rangle } =\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)\hat{D}_{0} \left(\hat{P_{n}}\otimes I\right)^{\dagger} \hat{R_{n+1}}^{\dagger}$$

Where $\hat{D}_{0}$ is a reflection around the perpendicular state to $|0\rangle_{n}$

$$\hat{D}_{0} = \hat{I}-2|0\rangle \langle0|$$

#### 3.2.1 Implementación  $\hat{D}_{0}$

The equivalent circuit for $\hat{D}_{0}$ is:

$$\hat{D}_{0} = \hat{I}-2|0\rangle \langle0|= \hat{X}^{\otimes n} c^{n-1}Z \hat{X}^{\otimes n}$$

The *AbstractGate* **D0_Gate** from **PhaseAmplification_Module** allow us do the operation:

In [ ]:
qLprog, P_Gate, R_gate = ExpectationLoadingData(p_X, f_X)
#Initial State
circuit = qLprog.to_circ()
job = circuit.to_job()
result = RunJob(lineal_qpu.submit(job))
InitialState = PostProcessResults(result)
#Applied Gate
from PhaseAmplification_Module import D0_Gate
D0 = D0_Gate(qLprog.qbit_count)
qLprog.apply(D0, qLprog.registers)

%qatdisplay D0 --depth 1
circuit = qLprog.to_circ()
%qatdisplay circuit

job = circuit.to_job()
result = RunJob(lineal_qpu.submit(job))
D0_State = PostProcessResults(result)

Operator $\hat{D}_{0}$ over state  $|\Psi\rangle_{n+1}$:

$$\hat{D}_{0}|\Psi\rangle_{n+1} = \hat{I}|\Psi\rangle_{n+1}-2|0\rangle \langle0|\Psi\rangle_{n+1}$$

It can be demostrated that the only difference between $\hat{D}_{0}|\Psi\rangle_{n+1}$ and $|\Psi\rangle_{n+1}$ is that the component $|0_{n}\rangle$ change the sign!!

In [ ]:
#Testing
C01 = np.isclose(InitialState['Amplitude'].loc[0], -D0_State['Amplitude'].loc[0])
C02 = np.isclose(InitialState['Amplitude'].loc[1:], D0_State['Amplitude'].loc[1:]).all()
print(C01 and C02)

#### 3.2.2 Implementation  $\hat{U}_{|\Psi}\rangle$

Finally the function **U_Phi_Gate** from **PhaseAmplification_Module** creates a customized AbstractGate that implements $\hat{U}_{|\Psi\rangle }$. This functions needs the initial $\mathcal{P}$ y $\mathcal{R}$ gates used for loading the data.

This operator is reflection around the ortoghonal state to $|\Psi\rangle_{n+1}$

In [ ]:
qLprog, P_Gate, R_gate = ExpectationLoadingData(p_X, f_X)
#Initial State
circuit = qLprog.to_circ()
job = circuit.to_job()
result = RunJob(lineal_qpu.submit(job))
InitialState = PostProcessResults(result)
#Applied Gate
from PhaseAmplification_Module import Load_UPhi_Gate
UPhi = Load_UPhi_Gate(P_Gate, R_gate)
qLprog.apply(UPhi, qLprog.registers)

%qatdisplay UPhi --depth 1
circuit = qLprog.to_circ()
%qatdisplay circuit

job = circuit.to_job()
result = RunJob(lineal_qpu.submit(job))
UPhi_State = PostProcessResults(result)

In this case 
$$\hat{U}_{|\Psi\rangle } |\Psi\rangle_{n+1} = \hat{I}|\Psi\rangle_{n+1} - 2|\Psi\rangle \langle \Psi|\Psi\rangle_{n+1}$$

$$\hat{U}_{|\Psi\rangle } |\Psi\rangle_{n+1} = |\Psi\rangle_{n+1} - 2|\Psi\rangle_{n+1} = -|\Psi\rangle_{n+1}$$



In [ ]:
#Testing
np.isclose(UPhi_State['Amplitude'], - InitialState['Amplitude']).all()